In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np
import random
import keras
import keras.backend as K

# Load your text file
filename = 'artofwar.txt'
with open(filename, encoding="utf8") as f:
    text = f.read().lower()

# Tokenization (at word level)
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index) + 1

# Convert text data into sequence of tokens
input_sequences = []
for line in text.split('.'):  # Assuming sentences end with a period
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences for equal length
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# Predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
label = keras.utils.to_categorical(label, num_classes=total_words)

def perplexity(y_true, y_pred):
    cross_entropy = K.categorical_crossentropy(y_true, y_pred)
    perplexity = K.exp(cross_entropy)
    return perplexity

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Embedding(total_words, 100, input_length=max_sequence_len - 1)) #100-dimensional word vectors
model.add(keras.layers.Bidirectional(keras.layers.LSTM(150, return_sequences=True))) # Can stack LSTMs if you want deeper understanding
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(100)))
model.add(keras.layers.Dense(total_words, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', perplexity])

In [ ]:
model.fit(predictors, label, batch_size=64, epochs=200, verbose=1)

Epoch 1/200
158/158 [==============================] - 27s 112ms/step - loss: 6.5421 - accuracy: 0.0621 - perplexity: 4428.5273
Epoch 2/200
158/158 [==============================] - 9s 54ms/step - loss: 6.0511 - accuracy: 0.0719 - perplexity: 2959.7747
Epoch 3/200
158/158 [==============================] - 5s 34ms/step - loss: 5.8368 - accuracy: 0.0962 - perplexity: 2420.8296
Epoch 4/200
158/158 [==============================] - 5s 30ms/step - loss: 5.6776 - accuracy: 0.1081 - perplexity: 1883.3098
Epoch 5/200
158/158 [==============================] - 5s 29ms/step - loss: 5.5251 - accuracy: 0.1240 - perplexity: 1503.2175
Epoch 6/200
158/158 [==============================] - 3s 21ms/step - loss: 5.3811 - accuracy: 0.1347 - perplexity: 1285.1851
Epoch 7/200
158/158 [==============================] - 4s 26ms/step - loss: 5.2600 - accuracy: 0.1413 - perplexity: 1091.8126
Epoch 8/200
158/158 [==============================] - 4s 24ms/step - loss: 5.1441 - accuracy: 0.1518 - perplexity: 

In [ ]:
def generate_text(seed_text, next_words, model, max_sequence_len, tokenizer):
    for _ in range(next_words):
        # Convert the current seed text to token sequence
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')

        # Predict probabilities for each word
        probabilities = model.predict(token_list, verbose=0)[0]

        # Choose the word with the highest probability as the next word
        predicted_index = np.argmax(probabilities)
        output_word = ""

        for word, index in tokenizer.word_index.items():
            if index == predicted_index:
                output_word = word
                break

        seed_text += " " + output_word

    return seed_text

# Use the function to generate text
seed_text = "Enemy must not be"
next_words = 50  # Number of words you want to generate
generated_text = generate_text(seed_text, next_words, model, max_sequence_len, tokenizer)
print(generated_text)